#### There should be a day for each panel. Make sure there is an hour for each day

In [13]:
import pandas as pd
import numpy  as np
import time
from config import getClient
from checkMissing import checkMissing
from os import listdir
from os.path import isfile, join

In [14]:
dataloc = getClient("OutFront")

In [15]:
panels = [f for f in listdir(dataloc+"panels/") if isfile(join(dataloc+"panels/", f))]

print("{:,.0f} panels to process".format(len(panels)))

699 panels to process


In [16]:
# A panel is missing hours
def addHours(key, missing):
    d = {}
    d["panel"]      = key[0]
    d["date"]       = key[1]
    d["population"] = None
    d["hour"]       = missing
    return pd.DataFrame.from_dict(d)

In [17]:
# Check each panel for missing dates
def processPanel(df):
    df.set_index(["panel", "date"], inplace=True)
    grp = df.groupby(level=[0,1])
    for key, val in grp:
        missing = checkMissing(df, "hour")
        if len(missing) > 0:
            imputed = addHours(key, missing)
            df      = df.append(imputed)
    return df

In [ ]:
count = 0
start = time.time()

for panel in panels:
    df = pd.read_csv(dataloc+"panels/"+panel,\
                     dtype={"hour":'int8',\
                            "population":'int32'})
    df = processPanel(df)
    df.to_csv(dataloc+"panels/"+panel)

end = time.time()
print("Completed after {:.0f} minutes".format((end-start)/60))